In [1]:
import sys     ###kldglobal （下のセルの3行目まで）
sys.path.append('../scripts/')
from mcl_global import *        #mcl_globalを読み込む
from scipy.stats import chi2

In [2]:
class KldMcl(Mcl):  
    def __init__(self, envmap, init_pose, max_num, motion_noise_stds, distance_dev_rate=0.14, direction_dev=0.05, 
                widths = np.array([0.2, 0.2, math.pi/18]).T, epsilon=0.1, delta=0.01):
        super().__init__(envmap, init_pose, max_num, motion_noise_stds, distance_dev_rate, direction_dev) #パーティクル数をmax_numに
        self.widths = widths                #各ビンのxyθの幅
        self.max_num = max_num     #パーティクル数の上限
        self.epsilon = epsilon              #ε
        self.delta = delta                    #δ
        self.binnum = 0         #ビンの数k。本来、ローカルの変数で良いけど描画用にオブジェクトに持たせておく
        
        self.observed = False
        
    def motion_update(self, nu, omega, time):  
        if not self.observed and len(self.particles) == self.max_num:
            for p in self.particles: p.motion_update(nu, omega, time, self.motion_noise_rate_pdf)
            return
            
        ws = [e.weight for e in self.particles]    # 重みのリストを作る
        if sum(ws) < 1e-100: ws = [e + 1e-100 for e in ws]  #重みの和がゼロに丸め込まれるとサンプリングできなくなるので小さな数を足しておく
        
        new_particles = [] #新しいパーティクルのリスト（最終的にself.particlesになる）
        bins = set()             #ビンのインデックスを登録しておくセット
        for i in range(self.max_num):
            chosen_p = random.choices(self.particles, weights=ws)  #1つだけ選ぶ（リストに1個だけ入っている）
            p = copy.deepcopy(chosen_p[0])
            p.motion_update(nu, omega, time, self.motion_noise_rate_pdf)  #移動
            bins.add(tuple(math.floor(e) for e in p.pose/self.widths))               #ビンのインデックスをsetに登録（角度を正規化するとより良い）
            new_particles.append(p)                                                                       #新しいパーティクルのリストに追加
            
            self.binnum = len(bins) if len(bins) > 1 else 2     #ビンの数が1の場合2にしないと次の行の計算ができない
            if  len(new_particles) > math.ceil(chi2.ppf(1.0 - self.delta, self.binnum-1)/(2*self.epsilon)):
                break
            
        self.particles = new_particles
        for i in range(len(self.particles)): #正規化
            self.particles[i].weight = 1.0/len(self.particles)
            
    def observation_update(self, observation):   
        for p in self.particles:
            p.observation_update(observation, self.map, self.distance_dev_rate, self.direction_dev) 
        self.set_ml()
#     self.resampling()             #motion_updateでリサンプリングするので削除
        self.observed = len(observation) > 0

    def draw(self, ax, elems):  
        super().draw(ax, elems)
        elems.append(ax.text(-4.5, -4.5, "paricle:{}, bin:{}".format(len(self.particles), self.binnum), fontsize=10)) #パーティクルとビンの数を表示

In [3]:
class KldMclAgent(MclAgent): 
    def __init__(self, time_interval, nu, omega, particle_pose, envmap, max_particle_num=1000, \
                motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}): #パーティクル数に関する引数名を変更
        super().__init__(time_interval, nu, omega, particle_pose, envmap, 1, motion_noise_stds) #self.pfはすぐ消すので1個だけで初期化
        self.pf = KldMcl(envmap, particle_pose, max_particle_num, motion_noise_stds) #MclからKldMclにself.pfの中身を入れ替え

In [4]:
def trial(animation):
    time_interval = 0.1
    world = World(30, time_interval, debug=not animation) 

    ## 地図を生成して2つランドマークを追加 ##  #一つランドマークを減らしておきましょう
    m = Map()
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)          

    ## ロボットを作る ##
    init_pose = np.array([np.random.uniform(-5.0, 5.0), np.random.uniform(-5.0, 5.0), np.random.uniform(-math.pi, math.pi)]).T
    
    a = KldMclAgent(time_interval, 0.2, 10.0/180*math.pi, None, m, max_particle_num=10000) #max_particle_numを設定
    r = Robot(init_pose, sensor=Camera(m), agent=a, color="red")
    world.append(r)

    world.draw()
#    world.ani.save('/tmp/anm.gif', writer='imagemagick', fps=1) #描画用
    
    return (r.pose, a.pf.ml.pose)

In [5]:
if __name__ == '__main__':
    ok = 0 ###mclglobal1exec
    for i in range(1000):
        actual, estm = trial(False)
        diff = math.sqrt((actual[0]-estm[0])**2 + (actual[1]-estm[1])**2)
        print(i, "真値:", actual, "推定値", estm, "誤差:", diff)
        if diff <= 1.0:
            ok += 1

    ok

0 真値: [-2.07145069 -0.81895588  6.87546111] 推定値 [-1.97991804 -0.43600881  6.76745237] 誤差: 0.39373427591941523
1 真値: [-3.75378672 -2.50253835  4.19113879] 推定値 [-2.62768031 -1.69623603  4.63707313] 誤差: 1.3850050886985514
2 真値: [ 0.42527791 -2.22939162  6.42245763] 推定値 [ 0.98049384 -1.83876506  6.10691882] 誤差: 0.6788621570626491
3 真値: [0.45740146 2.98993179 7.51522795] 推定値 [0.65781569 3.32078076 7.60676632] 誤差: 0.38681637576426514
4 真値: [-1.9743373  4.6920448  8.6909677] 推定値 [-2.25376797  4.46123735  8.50215259] 誤差: 0.36242733982792624
5 真値: [-1.73091696 -5.60842887  7.49287135] 推定値 [-2.58252992 -3.70182433  6.97914565] 誤差: 2.0881536080088425
6 真値: [-0.09502607  5.64875849  4.70571961] 推定値 [-1.3910808   4.52365454  4.85361915] 誤差: 1.7162799136137494
7 真値: [-0.65865532  5.05173753  2.95968435] 推定値 [-1.11921004  5.00979573  3.02003972] 誤差: 0.4624605544032217
8 真値: [5.5302998  3.43796748 2.41476648] 推定値 [5.56870716 3.78127971 2.5169264 ] 誤差: 0.3454539282160794
9 真値: [2.25152955 0.01293228 5.

/usr/local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


20 真値: [4.33108045 3.90139826 6.66769145] 推定値 [3.19589989 1.20629037 4.74653107] 誤差: 2.9244215527575332
21 真値: [-1.20134133 -2.29217539  7.63102789] 推定値 [-1.79214897 -1.9089071   7.5903827 ] 誤差: 0.7042359250650314
22 真値: [2.78112317 3.98502081 7.88587372] 推定値 [2.66188646 3.74348152 7.90902273] 誤差: 0.2693670745511344
23 真値: [3.59205604 3.61072148 2.98074924] 推定値 [3.72940181 3.35055548 2.62262673] 誤差: 0.29419416388652375
24 真値: [-1.86458154  0.3901117   5.14199676] 推定値 [-1.95931431  0.27646714  5.14344315] 誤差: 0.14795060809922833
25 真値: [-0.56387363 -4.2993242   7.57253522] 推定値 [-0.68313621 -4.4090426   7.62751555] 誤差: 0.16205459336869077
26 真値: [-1.04190745 -0.14524453  4.89713378] 推定値 [-1.17116915  0.05077298  4.91200712] 誤差: 0.23480087745315925
27 真値: [-0.149619    1.72174869  7.1794094 ] 推定値 [3.92377151e-03 1.88271014e+00 7.13914709e+00] 誤差: 0.22244993339918162
28 真値: [-4.61297519  6.80732308  4.09150747] 推定値 [0.84486616 2.10753407 2.31482414] 誤差: 7.202502961840174
29 真値: [-2.8995081

96 真値: [7.12003247 2.14317738 2.52677656] 推定値 [6.80476923 1.86374456 2.39475825] 誤差: 0.42127617663675676
97 真値: [0.74861296 1.72496017 5.25678273] 推定値 [0.9115037  1.38179023 5.27346174] 誤差: 0.37986708474143205
98 真値: [1.81942241 3.995622   2.15701171] 推定値 [1.79682439 4.33007364 2.0577668 ] 誤差: 0.3352142109290151
99 真値: [3.51615551 1.01352957 7.47641466] 推定値 [3.25802242 0.96525618 7.3714896 ] 誤差: 0.2626080887058935
100 真値: [-0.06297199 -1.93263815  3.40535453] 推定値 [ 3.28077065 -0.74800465  8.47841235] 誤差: 3.5473893701937906
101 真値: [-3.82447825  3.66004781  3.96313942] 推定値 [-3.47292747  3.62583184  3.79286311] 誤差: 0.3532119607400838
102 真値: [ 0.86285348 -3.65500589  8.03181687] 推定値 [ 2.91016673 -2.39493779  5.00467951] 誤差: 2.4040098117762976
103 真値: [1.53390413 0.99348086 6.74019141] 推定値 [1.19785915 1.37814609 6.60823075] 誤差: 0.5107774176003651
104 真値: [ 3.1302995  -2.63991175  4.47676193] 推定値 [ 3.32711004 -2.75207009  4.32246213] 誤差: 0.2265256696635937
105 真値: [ 1.98320737 -2.75586776 

171 真値: [-0.64978046  1.03204628  3.72496021] 推定値 [-0.39717574  0.94196196  3.68305672] 誤差: 0.26818711324549765
172 真値: [ 1.26878084 -2.73427228  2.76635131] 推定値 [ 2.95348947 -3.06112258  6.08801932] 誤差: 1.7161218683780897
173 真値: [-5.28107122  5.29985577  5.34221356] 推定値 [-3.11816089  5.03751037  4.70519469] 誤差: 2.1787625383224136
174 真値: [-1.14499873 -0.80271343  5.66942513] 推定値 [-0.94194153 -0.30221731  5.55291108] 誤差: 0.5401190594705689
175 真値: [ 4.66253294 -0.35963066  2.43897487] 推定値 [4.59821442 0.28769883 2.49454635] 誤差: 0.6505169768671953
176 真値: [4.7882355  3.66621708 2.90707183] 推定値 [5.10796466 3.40067062 2.784266  ] 誤差: 0.4156220135772674
177 真値: [2.86403459 4.51556149 6.5466088 ] 推定値 [4.41753473 3.01595986 4.91892152] 誤差: 2.1592053502892803
178 真値: [0.58255593 2.20061586 4.59117294] 推定値 [0.30112419 2.29917992 4.5756293 ] 誤差: 0.29819238087654293
179 真値: [1.43329066 0.98873308 8.7008935 ] 推定値 [1.14231762 0.5499123  2.30474564] 誤差: 0.5265253922017713
180 真値: [3.14632829 0.3483

246 真値: [ 1.27166757 -1.94023367  5.62095806] 推定値 [ 0.8543522  -2.25874683  6.07076914] 誤差: 0.5249788117474631
247 真値: [2.87807635 3.03995383 3.42368697] 推定値 [3.04255369 3.15922315 3.3346489 ] 誤差: 0.20316979787988543
248 真値: [ 0.13563128 -1.65949309  3.72807349] 推定値 [ 0.15213601 -1.29006703  3.77446738] 誤差: 0.36979455892893703
249 真値: [-4.94757355 -0.81378535  5.06461354] 推定値 [-5.43342201 -0.46180184  4.90355809] 誤差: 0.5999509315060073
250 真値: [ 2.01238035 -1.759245    7.55586008] 推定値 [ 1.70433286 -2.10828651  7.41842971] 誤差: 0.46553543516252105
251 真値: [-4.14334047 -1.59152741  4.82523493] 推定値 [-4.76012217 -1.56125882  4.29446871] 誤差: 0.617523966456318
252 真値: [-5.91301352 -3.55509412  4.69142623] 推定値 [-5.44330325 -3.0459081   5.27637633] 誤差: 0.6927468073409424
253 真値: [ 4.39826883 -0.69866406  3.07069955] 推定値 [ 4.12688714 -0.79115038  3.05497846] 誤差: 0.2867084658604331
254 真値: [-1.04543665  3.18738679  5.1554699 ] 推定値 [-1.48553821  2.93852379  4.88013102] 誤差: 0.5055909159846667
255 真

322 真値: [3.75421488 2.85657731 7.68816828] 推定値 [3.97353077 3.67795166 7.87822294] 誤差: 0.8501501482019004
323 真値: [-0.47712979 -1.91825658  2.01254804] 推定値 [ 0.32426342 -0.73447016  2.27205406] 誤差: 1.4295388714227912
324 真値: [-2.76151406 -0.70374676  3.03606979] 推定値 [-2.78509851 -0.54003517  2.99221353] 誤差: 0.1654016669060708
325 真値: [-0.03091705 -4.68010524  1.65844208] 推定値 [ 3.90037809 -2.51567628  4.35484389] 誤差: 4.487742670682826
326 真値: [ 2.58339744 -2.38119975  3.8042226 ] 推定値 [ 2.33102859 -2.10974782  4.22219825] 誤差: 0.3706429359949484
327 真値: [ 1.9128269  -4.61186969  7.03800078] 推定値 [ 3.53763646 -2.64254768  8.88113754] 誤差: 2.553083485420361
328 真値: [ 2.28584663 -0.79801977  7.08779907] 推定値 [ 2.34278557 -0.84341367  7.07358652] 誤差: 0.07281929007163644
329 真値: [-1.06491816 -1.79105937  5.48210017] 推定値 [-0.89697237 -1.42032868  5.35736998] 誤差: 0.4069975844094391
330 真値: [-0.1825782   3.7145253   8.08733043] 推定値 [0.01432038 3.06260036 8.44962068] 誤差: 0.68101041749219
331 真値: [-4.0

397 真値: [-1.10487441  2.21212695  3.44137531] 推定値 [-1.55360566  2.67305586  3.6117314 ] 誤差: 0.6432846949604709
398 真値: [-0.82056994 -2.7518105   5.57700319] 推定値 [-1.00924989 -3.38212766  5.70664415] 誤差: 0.6579512510116714
399 真値: [ 1.4735823  -1.34256869  4.52772658] 推定値 [ 1.46435874 -1.54373044  4.42122896] 誤差: 0.20137309661927488
400 真値: [-2.25776571  0.914768    5.42226492] 推定値 [-2.02812428  0.67557759  5.3141266 ] 誤差: 0.33158292965543384
401 真値: [ 0.38424855 -3.83038786  3.65479891] 推定値 [ 3.10790229 -1.62251159  6.92133341] 誤差: 3.5061385161651044
402 真値: [-0.85166761 -1.72179866  6.38660553] 推定値 [-0.78679364 -1.54919788  6.27464167] 誤差: 0.18438997906575308
403 真値: [-0.6869402   1.900614    6.90079668] 推定値 [-0.83414927  0.30096912  7.19217981] 誤差: 1.606404140714321
404 真値: [-2.65074237  3.22014311  4.50345353] 推定値 [-2.21519676  3.09890904  4.29471916] 誤差: 0.4521036157544577
405 真値: [-4.33401815 -3.65065532  6.44895336] 推定値 [-0.92689868 -1.62324754  8.41346205] 誤差: 3.9646999093388473

472 真値: [ 0.25599633 -0.57053361  7.42188386] 推定値 [ 0.63466003 -0.51438421  7.49499476] 誤差: 0.3828040676976225
473 真値: [1.74009387 1.49156494 4.45251172] 推定値 [2.49374381 2.16149052 4.32025174] 誤差: 1.0083593101468349
474 真値: [-1.32777458  1.28994636  5.15461224] 推定値 [-1.34437238  1.10647242  5.23503214] 誤差: 0.18422316030845098
475 真値: [ 2.0826709  -0.383917    4.78821832] 推定値 [ 2.34096921 -0.01688656  4.66496023] 誤差: 0.44880883071538313
476 真値: [1.56238075 0.76499654 5.02736229] 推定値 [1.35759218 0.5025576  5.037769  ] 誤差: 0.33288519522249366
477 真値: [-1.27726567 -4.02362455  5.60118548] 推定値 [ 5.5866526  -1.64457623  2.37769049] 誤差: 7.264519593517209
478 真値: [-1.75869556 -0.22491194  4.44803718] 推定値 [-1.79214146 -1.43896551  4.4449068 ] 誤差: 1.2145141829605328
479 真値: [1.45580922 1.51839384 2.72888853] 推定値 [1.24927647 0.82236464 8.84995091] 誤差: 0.7260250858307423
480 真値: [0.50561157 3.75395031 6.43606641] 推定値 [1.15112517 0.32615857 6.34929097] 誤差: 3.4880430086959335
481 真値: [ 4.45814749 -4

KeyboardInterrupt: 